In [ ]:
#Model 3- ANN -MLP Feedforward Network
# Load the dataset
path= "/content/sample_data/Anonymized Dataset for Info Science MS.xlsx"
df = pd.read_excel(path)

#Lets imput missing values as missing values %= 10860/62512= 17.37% which is significant portion of the data.
#Hence we will impute the missing values rather than discarding them


df['TermGPA'] = df['TermGPA'].fillna(df['TermGPA'].mean())
df['CumulativeGPA'] = df['CumulativeGPA'].fillna(df['CumulativeGPA'].mean())


#Drop insignificant column "FakeIdentifier"
df.drop('FakeIdentifier', axis=1, inplace=True)

#Drop the blank rows input
# Remove records where all three GPA components are zero
df = df[~((df['UnitsPassednotincludedinGPA'] == 0) &
          (df['NumberofClassesEnrolled'] == 0))]

#Lets encode categorical data and one hot encode nominal data

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#Encoding Nominal Predictors
nominal_cols = ['Gender', 'PrimaryMilitaryAffiliation', 'College',
                'UAFullTimePartTime', 'FirstGenerationFlag',"AcademicCareer"]
df = pd.get_dummies(df, columns=nominal_cols, drop_first=True)

#Converting Boolean columns  to int(0/1) type int data
bool_cols = df.select_dtypes(include="bool").columns
for col in bool_cols:
    df[col] = df[col].astype(int)

# Coding Ordinal Predictors
df["AcademicLevelEndofTerm"]=df["AcademicLevelEndofTerm"].replace({"Freshman": 1, "Sophomore": 2,"Junior":3,"Senior":4 ,"Graduate":5 ,"Masters":6 })
df["AcademicYear"]=df["AcademicYear"].replace({2020: 1, 2021: 2,2022:3,2023:4 ,2024:5 ,2025:6 })

#Lets drop highly correlated predictor variables
cols = ['College_James E Rogers College of Law', 'UnitsPassedincludedinGPA',"AcademicCareer_Undergraduate","UAFullTimePartTime_P"]

df.drop(columns=cols, inplace=True)

# Define the categorical, ordinal, and numerical columns
#categorical_cols = ['Gender', 'PrimaryMilitaryAffiliation', 'College','UAFullTimePartTime', 'FirstGenerationFlag',"AcademicCareer"]
#ordinal_cols = ["AcademicLevelEndofTerm", "AcademicYear"]
#numerical_cols = ["Age","NumberofClassesEnrolled","UnitsPassedincludedinGPA","UnitsPassednotincludedinGPA","TermGPA","CumulativeGPA"]


# Create a column transformer to preprocess the data
numeric_columns = ["Age","NumberofClassesEnrolled","UnitsPassednotincludedinGPA","TermGPA","CumulativeGPA","AcademicLevelEndofTerm", "AcademicYear"]
scaler = StandardScaler()
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])



# Define the predictor and output variables
X = df.drop(columns=["CumulativeGPA"])
y = df["CumulativeGPA"]

# Import necessary libraries
from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers import LeakyReLU
from keras.losses import Huber
from keras.layers import Dense, BatchNormalization, Dropout
from keras.regularizers import l2
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
from keras.optimizers import SGD




# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the data
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test = scaler_y.transform(y_test.values.reshape(-1, 1))
model = Sequential()

# New Hidden Layer 1 — 128 neurons
model.add(Dense(128, input_shape=(36,), kernel_regularizer=l2(0.001)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Hidden Layer 2 — 64 neurons
model.add(Dense(64, kernel_regularizer=l2(0.001)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Hidden Layer 3 — 32 neurons
model.add(Dense(32, kernel_regularizer=l2(0.001)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Hidden Layer 4 — 16 neurons
model.add(Dense(16, kernel_regularizer=l2(0.001)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())

# Output Layer — continuous regression output
model.add(Dense(1))

# Compile the Model
# -------------------------
model.compile(
    optimizer=Adam(learning_rate=0.0008),
    loss=Huber(delta=1.0),
    metrics=['mae', 'mse']
)

# Print the model summary
print(model.summary())

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
y_pred_inv = scaler_y.inverse_transform(y_pred)
y_test_inv = scaler_y.inverse_transform(y_test)

r2 = r2_score(y_test_inv, y_pred_inv)
#r2 = r2_score(y_test, y_pred)

print("Evaluation Metrics:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R-Squared (R2): {r2:.4f}")